<a href="https://colab.research.google.com/github/TirendazAcademy/Deep-Learning-with-TensorFlow/blob/main/How_to_Use_Keras_Functional_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building the Model for One Input & Output

## Loading dataset

In [1]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

## Data Preprocessing

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    housing.data, housing.target, random_state=42
)

In [3]:
import tensorflow as tf
tf.random.set_seed(42)

## Modeling

### Creating the layers

In [4]:
normalization_layer = tf.keras.layers.Normalization()
hidden_layer1 = tf.keras.layers.Dense(30, activation="relu")
hidden_layer2 = tf.keras.layers.Dense(30, activation="relu")
concat_layer = tf.keras.layers.Concatenate()
output_layer = tf.keras.layers.Dense(1)

### Building the model

In [5]:
input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
normalized = normalization_layer(input_)
hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([normalized, hidden2])
output = output_layer(concat)
model = tf.keras.Model(inputs=[input_], outputs=[output])

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 normalization (Normalization)  (None, 8)            17          ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 30)           270         ['normalization[0][0]']          
                                                                                                  
 dense_1 (Dense)                (None, 30)           930         ['dense[0][0]']                  
                                                                                              

## Model Training

In [7]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
normalization_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2)

Epoch 1/20
387/387 [==============================] - 8s 4ms/step - loss: 1.2202 - root_mean_squared_error: 1.1046 - val_loss: 0.5766 - val_root_mean_squared_error: 0.7593
Epoch 2/20
387/387 [==============================] - 1s 4ms/step - loss: 0.4747 - root_mean_squared_error: 0.6890 - val_loss: 0.4651 - val_root_mean_squared_error: 0.6820
Epoch 3/20
387/387 [==============================] - 1s 4ms/step - loss: 0.4065 - root_mean_squared_error: 0.6376 - val_loss: 0.4246 - val_root_mean_squared_error: 0.6516
Epoch 4/20
387/387 [==============================] - 2s 5ms/step - loss: 0.3821 - root_mean_squared_error: 0.6182 - val_loss: 0.3983 - val_root_mean_squared_error: 0.6311
Epoch 5/20
387/387 [==============================] - 2s 5ms/step - loss: 0.3711 - root_mean_squared_error: 0.6092 - val_loss: 0.3973 - val_root_mean_squared_error: 0.6303
Epoch 6/20
387/387 [==============================] - 2s 4ms/step - loss: 0.3706 - root_mean_squared_error: 0.6088 - val_loss: 0.3794 - val_

## Model Evaluation

In [8]:
mse_test = model.evaluate(X_test, y_test)

162/162 [==============================] - 0s 3ms/step - loss: 0.3089 - root_mean_squared_error: 0.5558


## Prediction

In [9]:
import pandas as pd
X_new = X_test[:3]
pd.DataFrame(X_new)

,0,1,2,3,4,5,6,7
0,1.6812,25.0,4.192201,1.022284,1392.0,3.877437,36.06,-119.01
1,2.5313,30.0,5.039384,1.193493,1565.0,2.679795,35.14,-119.46
2,3.4801,52.0,3.977155,1.185877,1310.0,1.360332,37.80,-122.44


In [10]:
y_pred = model.predict(X_new)
pd.DataFrame(y_pred)

1/1 [==============================] - 0s 107ms/step


,0
0,0.631996
1,1.136796
2,4.315046


In [11]:
pd.DataFrame(y_test[:3])

,0
0,0.47700
1,0.45800
2,5.00001


# Building the Model with Two Inputs & One Output

## Creating the layers

In [12]:
# Input layers
input_wide = tf.keras.layers.Input(shape=[5])
input_deep = tf.keras.layers.Input(shape=[6])

# Normalization layers
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()

## Modeling

In [13]:
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

# Hidden layers
hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)

# Concatenation layer
concat = tf.keras.layers.concatenate([norm_wide, hidden2])

#Output layer
output = tf.keras.layers.Dense(1)(concat)

# Model building
model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output])

## Model Compiling

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]

norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

history = model.fit((X_train_wide, X_train_deep), y_train, epochs=20, validation_split=0.2)

Epoch 1/20
387/387 [==============================] - 3s 4ms/step - loss: 1.3652 - root_mean_squared_error: 1.1684 - val_loss: 0.5692 - val_root_mean_squared_error: 0.7544
Epoch 2/20
387/387 [==============================] - 2s 4ms/step - loss: 0.4803 - root_mean_squared_error: 0.6930 - val_loss: 0.4685 - val_root_mean_squared_error: 0.6845
Epoch 3/20
387/387 [==============================] - 2s 4ms/step - loss: 0.4262 - root_mean_squared_error: 0.6528 - val_loss: 0.4444 - val_root_mean_squared_error: 0.6667
Epoch 4/20
387/387 [==============================] - 2s 4ms/step - loss: 0.4051 - root_mean_squared_error: 0.6365 - val_loss: 0.4323 - val_root_mean_squared_error: 0.6575
Epoch 5/20
387/387 [==============================] - 3s 7ms/step - loss: 0.3968 - root_mean_squared_error: 0.6299 - val_loss: 0.4270 - val_root_mean_squared_error: 0.6535
Epoch 6/20
387/387 [==============================] - 2s 6ms/step - loss: 0.3916 - root_mean_squared_error: 0.6258 - val_loss: 0.4039 - val_

## Model Evaluation

In [15]:
mse_test = model.evaluate((X_test_wide, X_test_deep), y_test)

162/162 [==============================] - 0s 3ms/step - loss: 0.3282 - root_mean_squared_error: 0.5729


In [16]:
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]
y_pred = model.predict((X_new_wide, X_new_deep))
pd.DataFrame(y_pred)

1/1 [==============================] - 0s 83ms/step


,0
0,0.494242
1,1.256057
2,3.799639


# Building the Model with Two Inputs & Two Outputs

### Model Building

In [17]:
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

# Hidden layers
hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)

# Concatenation layer
concat = tf.keras.layers.concatenate([norm_wide, hidden2])

#Output layers
output = tf.keras.layers.Dense(1)(concat)
aux_output = tf.keras.layers.Dense(1)(hidden2)

model = tf.keras.Model(inputs=[input_wide, input_deep],outputs=[output, aux_output])

## Model Compiling

In [18]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=("mse", "mse"), loss_weights=(0.9, 0.1), optimizer=optimizer,metrics=["RootMeanSquaredError"])

### Model Training

In [19]:
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

history = model.fit((X_train_wide, X_train_deep), (y_train, y_train), epochs=20, validation_split=0.2)

Epoch 1/20
387/387 [==============================] - 4s 6ms/step - loss: 1.6741 - dense_8_loss: 1.6575 - dense_9_loss: 1.8238 - dense_8_root_mean_squared_error: 1.2874 - dense_9_root_mean_squared_error: 1.3505 - val_loss: 0.7913 - val_dense_8_loss: 0.7826 - val_dense_9_loss: 0.8696 - val_dense_8_root_mean_squared_error: 0.8847 - val_dense_9_root_mean_squared_error: 0.9325
Epoch 2/20
387/387 [==============================] - 2s 5ms/step - loss: 0.6648 - dense_8_loss: 0.6548 - dense_9_loss: 0.7548 - dense_8_root_mean_squared_error: 0.8092 - dense_9_root_mean_squared_error: 0.8688 - val_loss: 0.6075 - val_dense_8_loss: 0.5973 - val_dense_9_loss: 0.6998 - val_dense_8_root_mean_squared_error: 0.7728 - val_dense_9_root_mean_squared_error: 0.8365
Epoch 3/20
387/387 [==============================] - 3s 7ms/step - loss: 0.5277 - dense_8_loss: 0.5165 - dense_9_loss: 0.6289 - dense_8_root_mean_squared_error: 0.7187 - dense_9_root_mean_squared_error: 0.7930 - val_loss: 0.5061 - val_dense_8_loss

### Model Evaluation

In [20]:
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results

162/162 [==============================] - 1s 4ms/step - loss: 0.3283 - dense_8_loss: 0.3221 - dense_9_loss: 0.3838 - dense_8_root_mean_squared_error: 0.5675 - dense_9_root_mean_squared_error: 0.6195


### Model Prediction

In [21]:
y_pred_main, y_pred_aux = model.predict((X_new_wide, X_new_deep))

1/1 [==============================] - 0s 99ms/step


In [22]:
y_pred_main

array([[0.55358607],
       [1.0681849 ],
       [3.742366  ]], dtype=float32)

In [23]:
y_pred_aux

array([[0.5900813],
       [0.9094662],
       [3.6510856]], dtype=float32)

In [24]:
y_pred_tuple = model.predict((X_new_wide, X_new_deep))
y_pred = dict(zip(model.output_names, y_pred_tuple))

1/1 [==============================] - 0s 33ms/step


In [25]:
y_pred

{'dense_8': array([[0.55358607],
        [1.0681849 ],
        [3.742366  ]], dtype=float32), 'dense_9': array([[0.5900813],
        [0.9094662],
        [3.6510856]], dtype=float32)}

Let's connect [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [Twitter](http://twitter.com/tirendazacademy) | [Instagram](https://www.instagram.com/tirendazacademy) |[GitHub](http://github.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) | [Kaggle](https://www.kaggle.com/tirendazacademy) 😎

# Resource

- [Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)
